# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8],\
                         row[12], row[13], row[16]))


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Inserting data into Cassandra

#### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [5]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


## Modeling the data

## Create queries to ask the following three questions of the data:

## Query #1

### Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

SQL form: `SELECT artist, song, song_lenght FROM session_songs WHERE session_id=338 AND item_in_session=4`

By the nature of this query, the table must contain `artist`, `song` and `song_lenght`.
The primary key must be composed of `session_id` with `item_in_session` as clustering column to uniquely define rows.

In [7]:
query_session_songs = 'SELECT artist, song, song_lenght FROM session_songs WHERE session_id=338 AND item_in_session=4'                

#### Creating table session_songs

In [8]:
drop_session_songs = 'DROP TABLE IF EXISTS session_songs'
create_session_songs = 'CREATE TABLE IF NOT EXISTS session_songs \
                        (session_id int, item_in_session int, artist text, \
                        song text, song_lenght float, PRIMARY KEY (session_id, item_in_session))'

try:
    session.execute(drop_session_songs)
    session.execute(create_session_songs)
except e:
    print(e)

#### Inserting data into session_songs

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO session_songs \
                (session_id, item_in_session, artist, song, song_lenght) VALUES "
        query = query + "(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
    f.close()

#### Checking data for session_songs

In [10]:
try:
    rows = session.execute(query_session_songs)
    for row in rows:
        print (row.artist, row.song, row.song_lenght)
except Exception as e:
    print(e)
    


Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query #2

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

SQL form: `SELECT artist, song, user_first_name, user_last_name FROM user_sessions WHERE user_id=10 AND session_id=182`

By the nature of this query, the table must contain `artist`, `song` and both `first_name` and `last_name`.
The primary key must be composed by `user_id` and `session_id`, with `item_in_session` as clustering column for sorting.

In [11]:
query_user_songs = 'SELECT artist, song, user_first_name, user_last_name FROM user_songs WHERE user_id=10 AND session_id=182'

#### Creating table user_songs

In [12]:
drop_user_songs = 'DROP TABLE IF EXISTS user_songs'
create_user_songs = 'CREATE TABLE IF NOT EXISTS user_songs \
                (user_id int, session_id int, item_in_session int, artist text, \
                song text, user_first_name text, user_last_name text, \
                PRIMARY KEY ((user_id, session_id), item_in_session))'

try:
    session.execute(drop_user_songs)
    session.execute(create_user_songs)
except e:
    print(e)

#### Inserting data into user_songs

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO user_songs \
                (user_id, session_id, item_in_session, artist, song, \
                user_first_name, user_last_name) VALUES "
        query = query + "(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], 
                               line[9], line[1], line[4]))
    f.close()

#### Checking data for user_songs

In [14]:
try:
    rows = session.execute(query_user_songs)
    for row in rows:
        print (row.artist, row.song, row.user_first_name, row.user_last_name)
except Exception as e:
    print(e)
    


Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query #3

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

SQL form: `SELECT artist, song, song_lenght FROM session_songs WHERE session_id=338 AND item_in_session=4`

By the nature of this query, the table must contain `user_first_name` and `user_last_name`.
To uniquely identify a row, the table uses the composite key of `song_title` and `user_id`.

In [19]:
users_by_song = "SELECT user_first_name, user_last_name FROM users_by_song WHERE song='All Hands Against His Own'"                    

#### Creating table users_by_song

In [16]:
drop_users_by_song = 'DROP TABLE IF EXISTS users_by_song'
create_users_by_song = 'CREATE TABLE IF NOT EXISTS users_by_song \
                (song text, user_id int, user_first_name text, user_last_name text, PRIMARY KEY (song, user_id))'

try:
    session.execute(drop_users_by_song)
    session.execute(create_users_by_song)
except e:
    print(e)

#### Inserting data into users_by_song

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO users_by_song \
                (song, user_id, user_first_name, user_last_name) VALUES "
        query = query + "(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
    f.close()

#### Checking data for users_by_song

In [20]:
try:
    rows = session.execute(users_by_song)
    for row in rows:
        print (row.user_first_name, row.user_last_name)
except Exception as e:
    print(e)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [22]:
try:
    session.execute(drop_session_songs)
    session.execute(drop_user_songs)
    session.execute(drop_users_by_song)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()